# Zeitserien

Eine sehr spezielle Art von Daten sind sog. *Zeitserien*. Hier wird ein (oder mehrere) Messwert(e) über die Zeit aufgetragen. Die Vorhersage kannst du hier als *zeitliche* Vorhersage betrachten, d.h. es hat eine gewisse Verbindung zur Wettervorhersage - auch das Wetter könnte man als Zeitserie auffassen.

## Datenset laden

Am besten bleibst du gleich beim Wetter. Als Datenset verwendest du die täglichen Minimaltemperaturen in Melbourne. Eine genauere Erklärung über das Datenset und auch über Zeitserien findest du auf https://machinelearningmastery.com/time-series-datasets-for-machine-learning/

Das Datenset lädst du am besten von Github:

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv",
                 parse_dates=["Date"])
df

Du kannst dir gut einen Überblick verschaffen, indem du das Datenset als Diagramm anzeigen lässt:

In [ ]:
df.set_index("Date").plot(figsize=(16, 9))

## Lineare Regression mit Trainings- und Test-Daten

Zunächst versuchst du es mit linearer Regression abzubilden. Das Verfahren kennst du schon - hier gibt es eine kleine Änderung, weil nur eine Variable als unabhängige Variable zur Verfügung steht. Weil `sklearn` nicht gut mit dem Datum als Variable umgehen kann, nutzt du einfach einen `range`:

In [ ]:
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split([[v] for v in range(len(df))], 
                                                      df["Temp"].values)

Die Regression `fit`test du nur mit den Trainingsdaten

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

## Fehler berechnen

In diesem Fall reicht es, wenn du einen Score berechnest. Nutze den `r2`-Score:

In [ ]:
from sklearn import metrics
metrics.r2_score(y_train, lr.predict(X_train)), metrics.r2_score(y_test, lr.predict(X_test))

Selbst der Score der Trainingsdaten sieht gar nicht gut aus. Woran liegt das? Du kannst die Vorhersage durchführen und dir die Daten anzeigen lassen:

In [ ]:
df["Predicted Temp"] = lr.predict([[v] for v in range(len(df))])

In [ ]:
df.set_index("Date")[["Temp", "Predicted Temp"]].plot(figsize=(16, 9))

Die Regression hat einfach den Mittelwert vorhergesagt und die jahrezeitlichen Schwankungen gar nicht berücksichtigt. Das funktioniert so also nicht. Du brauchst ein besseres Verfahren

## Vorhersage von Zeitreihen

Mit `sklearn` kannst du auch mehrdimensionale (oder sog. *multivariate*) Regression durchführen. Es ist gar nicht einfach, die richtigen Features zu finden. In diesem Beispiel benutzt du einfach *alle*:

In [ ]:
!pip install prophet

In [ ]:
from prophet import Prophet

`prophet` erfordert einen `DataFrame`, der aus den beiden Spalten `ds` und `y` besteht:

In [ ]:
pdf = pd.DataFrame({"ds": df["Date"].values, 
                    "y": df["Temp"].values})
pdf

Anschließend kannst du ein `Prophet`-Objekt instanziieren und die Daten - ganz ähnlich wie bei `sklearn` *fitten*:

In [ ]:
m = Prophet()
m.fit(pdf)

`Prophet` hat nun aus den vergangenen Daten gelernt. Du möchtest eine Vorhersage für die zukünftigen Daten durchführen. Dazu erzeugt du einen weiteren `DataFrame`, der von hier ab in die Zukunft geht. Zwei Jahre entsprechen (meistens) 730 Tagen, als Periode gibst du "Tag" (`D`) an:

In [ ]:
future = m.make_future_dataframe(periods=730, freq='D')
future

Du siehst, dass der `DataFrame` nur bis zum 30.12.1992 läuft - 1992 war ein Schaltjahr. Das stört dich (hier) nicht.

Jetzt kannst du `Prophet` die Vorhersage für die Zukunft durchführen und dir die letzten Ergebnisse anzeigen lassen:

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

Das sieht gut aus.

Ein großer Vorteil von `Prophet` sind die Visualisierungen, die es bereits eingebaut hat:

In [ ]:
fig1 = m.plot(forecast)

Die Vorhersage sieht gut aus, auch die Interpolation der vergangenen Werte.

`Prophet` kann noch mehr und auch längerfristige Trends anzeigen sowie Periodiziäten:

In [ ]:
fig2 = m.plot_components(forecast)

Ab 1983 war ein leichter Temperaturanstieg zu beobachten, der sich 1987 verschärft hat. Mittwochs war es im Mittel am wärmsten, aber der Unterschied von 0,3°C ist nicht signifikant. Eher schon der Monatstrend, den du beobachten kannst. Auf der Südhalbkugel ist es im Juli am kältesten, immerhin 8°C vergleichen mit Februar. Verkehrte Welt.

## Vorhersage von Zeitserien

Wie du gesehen hast, funktioniert eine lineare Regression bei Zeitserien (oft) nicht gut. Das liegt an Periodizitäten und daran, dass Daten aus der unmittelbaren und längeren Vergangenheit berücksichtigt werden müssen. Das macht die Vorhersage zu einer echten Kunst.

Es gibt auch sog. ARIMA-Modelle, mit denen man gute Ergebnisse erzielen kann. Diese haben wir hier nicht betrachtet, da die Machine Learning-Modelle in der Zwischenzeit mindestens genauso gut funktionieren und nicht so komplex getuned werden müssen.

`Prophet` macht seine Sache sehr gut. Im Hintergrund werkeln dabei verschiedene (auch komplexe) Verfahren wie LSTMs usw., die du aber gar nicht zu Gesicht bekommst. Dafür sind die Ergebnisse wirklich gut!